In [11]:
import requests, zipfile, io, os

# Tự động tải và giải nén bộ dữ liệu MovieLens vào thư mục hiện tại
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
if not os.path.exists('ml-latest-small'):
    print("🚀 Đang tải dữ liệu MovieLens...")
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    print("✅ Đã giải nén xong!")

🚀 Đang tải dữ liệu MovieLens...
✅ Đã giải nén xong!


In [12]:
import pandas as pd

# Đọc file ratings.csv (đã có header sẵn: userId, movieId, rating, timestamp)
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# Kiểm tra dữ liệu
print("Kích thước file ratings:", ratings.shape)
ratings.head()

Kích thước file ratings: (100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
# Đọc file movies.csv
movies = pd.read_csv('ml-latest-small/movies.csv')

# Không đổi tên cột nữa, giữ nguyên: movieId, title, genres
print("Dữ liệu phim (giữ nguyên movieId):")
print(movies.columns)
movies.head()

Dữ liệu phim (giữ nguyên movieId):
Index(['movieId', 'title', 'genres'], dtype='object')


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
import pandas as pd

# -----------------------------------------------------------
# PHẦN 1: XỬ LÝ TAGS (Giữ nguyên logic cũ của bạn)
# -----------------------------------------------------------
# 1. Đọc file tags.csv
tags = pd.read_csv('ml-latest-small/tags.csv')

# 2. Chuẩn hóa tag
tags['tag'] = tags['tag'].astype(str).str.lower().str.strip()

# 3. Lọc tag hiếm
tag_counts = tags['tag'].value_counts()
valid_tags = tag_counts[tag_counts >= 2].index
tags = tags[tags['tag'].isin(valid_tags)]

# 4. Gộp các tag của cùng một bộ phim
movie_tags = (
    tags.groupby('movieId')['tag']
    .apply(lambda x: ' '.join(x))
    .reset_index()
)

# -----------------------------------------------------------
# PHẦN 2: TÍNH RATING TRUNG BÌNH (MỚI THÊM VÀO) ⭐
# -----------------------------------------------------------
# Tính điểm trung bình và số lượt vote cho mỗi phim từ bảng 'ratings' (đã load ở Cell 3)
movie_stats = ratings.groupby('movieId').agg({'rating': ['mean', 'count']})

# Làm phẳng MultiIndex (gộp tên cột)
movie_stats.columns = ['rating', 'vote_count'] # Đặt tên là 'rating' để khớp với code Backend
movie_stats = movie_stats.reset_index()

# -----------------------------------------------------------
# PHẦN 3: GHÉP TẤT CẢ VÀO BẢNG MOVIES
# -----------------------------------------------------------
# 1. Ghép Tags
movies = pd.merge(movies, movie_tags, on='movieId', how='left')

# 2. Ghép Ratings (MỚI)
movies = pd.merge(movies, movie_stats, on='movieId', how='left')

# 3. Xử lý dữ liệu thiếu (NaN) sau khi merge
movies['tag'] = movies['tag'].fillna('')
movies['rating'] = movies['rating'].fillna(0) # Phim chưa ai vote thì cho 0 điểm
movies['vote_count'] = movies['vote_count'].fillna(0)

# 4. Làm tròn điểm số (cho đẹp) - lấy 1 số thập phân (VD: 3.6)
movies['rating'] = movies['rating'].round(2)

# -----------------------------------------------------------
# PHẦN 4: TẠO METADATA SOUP (Giữ nguyên)
# -----------------------------------------------------------
# Xử lý genres
movies['genres_text'] = (
    movies['genres']
    .str.replace('|', ' ', regex=False)
    .str.lower()
    .replace('no genres listed', '')
)

# Tạo metadata soup
movies['metadata_soup'] = (
    movies['genres_text'] + ' ' + movies['tag']
).str.strip()

# Kiểm tra kết quả
print("Dữ liệu sau khi xử lý (đã có rating):")
print(movies[['title', 'rating', 'vote_count', 'tag']].head())

Dữ liệu sau khi xử lý (đã có rating):
                                title  rating  vote_count  \
0                    Toy Story (1995)    3.92       215.0   
1                      Jumanji (1995)    3.43       110.0   
2             Grumpier Old Men (1995)    3.26        52.0   
3            Waiting to Exhale (1995)    2.36         7.0   
4  Father of the Bride Part II (1995)    3.07        49.0   

                      tag  
0         pixar pixar fun  
1  fantasy robin williams  
2                          
3                          
4        pregnancy remake  


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sắp xếp lại index từ 0 -> N liên tục, xóa bỏ mọi lỗ hổng nếu có
movies = movies.reset_index(drop=True)

# 1. Khởi tạo TF-IDF với cấu hình tối ưu cho hệ gợi ý phim
tfidf = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),    # dùng cả unigram + bigram
    min_df=2,              # bỏ từ xuất hiện < 2 phim
    max_df=0.9             # bỏ từ xuất hiện quá nhiều
)

# 2. Học và biến đổi dữ liệu từ metadata_soup
tfidf_matrix = tfidf.fit_transform(movies['metadata_soup'])

# 3. Kiểm tra kích thước ma trận
print("Kích thước ma trận TF-IDF:", tfidf_matrix.shape)


Kích thước ma trận TF-IDF: (9742, 1385)


In [16]:
from sklearn.metrics.pairwise import linear_kernel

# linear_kernel nhanh hơn cosine_similarity với TF-IDF đã chuẩn hóa
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [17]:
# CELL 8: CHUẨN BỊ DỮ LIỆU & CHIA TRAIN/TEST
from sklearn.model_selection import train_test_split
import pandas as pd

# --- 1. LỌC NHIỄU (Pre-processing) ---
# Lọc bỏ những user đánh giá quá ít (dưới 5 phim) để tránh nhiễu
# Những user này thường không đủ dữ liệu để tính độ tương đồng chính xác
min_ratings = 5
user_counts = ratings['userId'].value_counts()
valid_users = user_counts[user_counts >= min_ratings].index
ratings_filtered = ratings[ratings['userId'].isin(valid_users)].copy()

print(f"Dữ liệu gốc: {len(ratings)} dòng")
print(f"Dữ liệu sau khi lọc (User >= {min_ratings} ratings): {len(ratings_filtered)} dòng")

# --- 2. CHIA DỮ LIỆU (Train 80% - Test 20%) ---
# Dùng cho quy trình đánh giá (Evaluation)
train_data, test_data = train_test_split(
    ratings_filtered,
    test_size=0.2,
    random_state=42
)

# Tạo bản copy để tránh cảnh báo SettingWithCopyWarning
train_data = train_data.copy()
test_data = test_data.copy()

print("\nĐã chia dữ liệu xong:")
print(f"- Tập Train (Evaluation): {len(train_data)} dòng")
print(f"- Tập Test (Evaluation):  {len(test_data)} dòng")
print(f"- Tập Full (Production):  {len(ratings_filtered)} dòng")

Dữ liệu gốc: 100836 dòng
Dữ liệu sau khi lọc (User >= 5 ratings): 100836 dòng

Đã chia dữ liệu xong:
- Tập Train (Evaluation): 80668 dòng
- Tập Test (Evaluation):  20168 dòng
- Tập Full (Production):  100836 dòng


In [18]:
# CELL 9: TÍNH TOÁN MA TRẬN & ĐỘ TƯƠNG ĐỒNG (MATRIX & SIMILARITY CALCULATION)
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

def create_cf_matrices(df_input, label="Data"):
    """
    Hàm nhận vào DataFrame (Train hoặc Full), thực hiện biến đổi dữ liệu để tạo ra 3 ma trận cốt lõi:
    1. user_mean: Series điểm trung bình của mỗi User.
    2. matrix: Ma trận thưa (User-Item) đã được chuẩn hóa (trừ đi trung bình).
    3. user_sim: Ma trận độ tương đồng Cosine giữa các User.
    """
    print(f"\n🚀 Đang tính toán ma trận cho tập: {label}...")

    # 1. Tính Mean Ratings (Thống kê)
    user_mean = df_input.groupby('userId')['rating'].mean()

    # 2. Centering (Chuẩn hóa dữ liệu)
    # Trừ điểm đánh giá cho điểm trung bình của user đó để loại bỏ bias (người khó tính/dễ tính)
    df_merge = df_input.merge(user_mean.rename('mean_rating'), on='userId', how='left')
    df_merge['rating_centered'] = df_merge['rating'] - df_merge['mean_rating']

    # 3. Pivot Table (Tạo ma trận thưa)
    # index=User, columns=Movie, values=Centered Rating
    matrix = df_merge.pivot_table(
        index='userId',
        columns='movieId',
        values='rating_centered'
    ).fillna(0)

    # 4. Tính Cosine Similarity (Đại số tuyến tính)
    # Tính góc giữa các vector user trong không gian đa chiều
    sim_matrix = cosine_similarity(matrix)

    # Chuyển kết quả từ numpy array sang DataFrame để dễ lưu trữ
    sim_df = pd.DataFrame(sim_matrix, index=matrix.index, columns=matrix.index)

    print(f"   ✓ Đã tạo Matrix kích thước: {matrix.shape}")
    print(f"   ✓ Đã tính xong độ tương đồng Cosine: {sim_df.shape}")

    return user_mean, matrix, sim_df

# --- CHẠY THỬ NGHIỆM (DRY RUN) ---
# Chạy thử với tập Train để đảm bảo code không lỗi logic trước khi lưu
print("--- Kiểm tra thử hàm tính toán ---")
_ = create_cf_matrices(train_data, label="Train Data Preview")

--- Kiểm tra thử hàm tính toán ---

🚀 Đang tính toán ma trận cho tập: Train Data Preview...
   ✓ Đã tạo Matrix kích thước: (610, 8983)
   ✓ Đã tính xong độ tương đồng Cosine: (610, 610)


In [19]:
# CELL 10: LƯU TRỮ DỮ LIỆU THEO CẤU TRÚC 3 FOLDER
import os
import joblib

# 1. TẠO CẤU TRÚC THƯ MỤC
base_path = 'data/processed'
paths = {
    'common': os.path.join(base_path, 'common'),
    'eval': os.path.join(base_path, 'evaluation'),
    'prod': os.path.join(base_path, 'production')
}

for p in paths.values():
    os.makedirs(p, exist_ok=True)

print(f"📁 Đã tạo cấu trúc folder tại: {base_path}")

# =================================================================
# A. FOLDER COMMON (Dùng chung cho cả 2)
# =================================================================
print("\n1️⃣ XỬ LÝ COMMON FILES (Content-Based)...")

# 1. Movies (Thông tin phim)
movies.to_csv(os.path.join(paths['common'], 'movies_cleaned.csv'), index=False, encoding='utf-8-sig')

# 2. TF-IDF & Cosine Similarity
# (Biến tfidf_matrix và cosine_sim lấy từ các cell Content-Based phía trước)
joblib.dump(tfidf_matrix, os.path.join(paths['common'], 'tfidf_matrix.pkl'))
joblib.dump(cosine_sim, os.path.join(paths['common'], 'cosine_similarity_matrix.pkl'))

print("   ✓ Đã lưu: movies, tfidf, cosine_sim vào /common")

# =================================================================
# B. FOLDER EVALUATION (Dùng cho chấm điểm - 80% Data)
# =================================================================
print("\n2️⃣ XỬ LÝ EVALUATION FILES (Train Data)...")

# 1. Lưu dữ liệu gốc để đối chiếu
train_data.to_csv(os.path.join(paths['eval'], 'train_data.csv'), index=False, encoding='utf-8-sig')
test_data.to_csv(os.path.join(paths['eval'], 'test_data.csv'), index=False, encoding='utf-8-sig')

# 2. Tính toán & Lưu Artifacts (Dựa trên Train)
mean_eval, matrix_eval, sim_eval = create_cf_matrices(train_data, label="Evaluation Data")

mean_eval.to_csv(os.path.join(paths['eval'], 'user_mean_ratings.csv'), header=['mean_rating'])
matrix_eval.to_csv(os.path.join(paths['eval'], 'user_item_matrix_centered.csv'))
joblib.dump(sim_eval, os.path.join(paths['eval'], 'user_similarity_matrix.pkl'))

print("   ✓ Đã lưu đầy đủ artifacts vào /evaluation")

# =================================================================
# C. FOLDER PRODUCTION (Dùng cho App - 100% Data)
# =================================================================
print("\n3️⃣ XỬ LÝ PRODUCTION FILES (Full Data)...")

# 1. Lưu dữ liệu full
ratings_filtered.to_csv(os.path.join(paths['prod'], 'ratings_cleaned.csv'), index=False, encoding='utf-8-sig')

# 2. Tính toán & Lưu Artifacts (Dựa trên Full Data)
mean_prod, matrix_prod, sim_prod = create_cf_matrices(ratings_filtered, label="Production Data")

mean_prod.to_csv(os.path.join(paths['prod'], 'user_mean_ratings.csv'), header=['mean_rating'])
matrix_prod.to_csv(os.path.join(paths['prod'], 'user_item_matrix_centered.csv'))
joblib.dump(sim_prod, os.path.join(paths['prod'], 'user_similarity_matrix.pkl'))

print("   ✓ Đã lưu đầy đủ artifacts vào /production")

print("\n🎉 HOÀN TẤT! Hệ thống file đã sẵn sàng.")

📁 Đã tạo cấu trúc folder tại: data/processed

1️⃣ XỬ LÝ COMMON FILES (Content-Based)...
   ✓ Đã lưu: movies, tfidf, cosine_sim vào /common

2️⃣ XỬ LÝ EVALUATION FILES (Train Data)...

🚀 Đang tính toán ma trận cho tập: Evaluation Data...
   ✓ Đã tạo Matrix kích thước: (610, 8983)
   ✓ Đã tính xong độ tương đồng Cosine: (610, 610)
   ✓ Đã lưu đầy đủ artifacts vào /evaluation

3️⃣ XỬ LÝ PRODUCTION FILES (Full Data)...

🚀 Đang tính toán ma trận cho tập: Production Data...
   ✓ Đã tạo Matrix kích thước: (610, 9724)
   ✓ Đã tính xong độ tương đồng Cosine: (610, 610)
   ✓ Đã lưu đầy đủ artifacts vào /production

🎉 HOÀN TẤT! Hệ thống file đã sẵn sàng.
